In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
file_path = '/content/drive/My Drive/ML project/phase2_students_before_cleaning.csv'
data = pd.read_csv(file_path)
df = data.copy()
df.head()

,flow_time,header_size,packet_duration,overall_rate,src_rate,dst_rate,fin_packets,urg_packets,rst_packets,max_value,...,syn_flags,rst_flags,psh_flags,ack_flags,protocol_http,protocol_https,protocol_tcp,protocol_udp,protocol_icmp,label
0,0.041268,15499.00,64.00,7805.845961,7805.845961,0.0,0.0,0.00,0.00,50.00,...,0,0,0,0,0,0,0,1,0,DDoS
1,0.018393,3702.54,64.00,6728.994198,6728.994198,0.0,0.0,0.00,0.01,54.28,...,0,0,0,0,0,0,0,1,0,DDoS
2,0.000000,182.00,64.00,38.559448,38.559448,0.0,0.0,0.00,0.00,182.00,...,0,0,0,0,0,0,0,1,0,DoS
3,0.109292,35027.55,62.72,6783.234241,6783.234241,0.0,0.0,0.03,0.11,65.11,...,0,0,0,0,0,0,0,1,0,DoS
4,0.000000,162.00,64.00,2.305494,2.305494,0.0,0.0,0.00,0.00,162.00,...,0,0,0,0,0,0,0,1,0,DoS


In [8]:
target = df['label']

In [9]:
target

,label
0,DDoS
1,DDoS
2,DoS
3,DoS
4,DoS
...,...
938578,DDoS
938579,DoS
938580,DoS
938581,DDoS


In [11]:
numerical_columns = df.select_dtypes(include=np.number).columns
for col in numerical_columns:
    if (df[col] < 0).any():
        df[col] = df[col].clip(lower=0)

In [13]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [14]:
df['label'] = label_encoder.fit_transform(df['label'])

In [15]:
target = df['label'].value_counts()

In [20]:
X = df.drop(columns=['label'])
y = df['label']

In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [24]:
from sklearn.preprocessing import QuantileTransformer

scaler = QuantileTransformer(output_distribution='normal')

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [26]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=42)

In [27]:
from sklearn.metrics import accuracy_score
y_pred_rf = rf_model.predict(X_test_scaled)
acc_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {acc_rf:.4f}")

Random Forest Accuracy: 0.9001


In [28]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    random_state=42,
    use_label_encoder=False,
    eval_metric='logloss'
)
xgb_model.fit(X_train_scaled, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [22:41:07] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...)

In [29]:
y_pred_xgb = xgb_model.predict(X_test_scaled)
acc_xgb = accuracy_score(y_test, y_pred_xgb)
print(f"XGBoost Accuracy: {acc_xgb:.4f}")

XGBoost Accuracy: 0.9111


In [30]:
test = pd.read_csv('/content/drive/My Drive/ML project/test.csv')

In [31]:
test_ids = test['Id']
test_features = test.drop(columns=['Id'])

# Same preprocessing on test set
for col in numerical_columns:
    if col in test_features.columns and (test_features[col] < 0).any():
        test_features[col] = test_features[col].clip(lower=0)

# Scale features
test_scaled = scaler.transform(test_features)

# 6. Predict and save submissions
# RF submission
preds_rf = rf_model.predict(test_scaled)
preds_rf_decoded = label_encoder.inverse_transform(preds_rf)
submission_rf = pd.DataFrame({'Id': test_ids, 'label': preds_rf_decoded})
submission_rf.to_csv(
    '/content/drive/My Drive/ML project/submission_random_forest2.csv',
    index=False
)
print("Random Forest submission file created: submission_random_forest.csv")

# XGB submission
preds_xgb = xgb_model.predict(test_scaled)
preds_xgb_decoded = label_encoder.inverse_transform(preds_xgb)
submission_xgb = pd.DataFrame({'Id': test_ids, 'label': preds_xgb_decoded})
submission_xgb.to_csv(
    '/content/drive/My Drive/ML project/submission_xgboost2.csv',
    index=False
)
print("XGBoost submission file created: submission_xgboost.csv")

Random Forest submission file created: submission_random_forest.csv
XGBoost submission file created: submission_xgboost.csv
